## 전처리 함수

In [1]:
# from nltk.stem import WordNetLemmatizer
import pandas as pd
import re
from fuzzywuzzy import fuzz
import numpy as np

df = pd.read_csv("../data/orginazation_label.csv", encoding='latin1')
df2 = pd.read_csv('../data/nasdaq_stocks.csv', encoding='latin1')
df3 = pd.read_csv('C:/projectnasdaq/nasdaq_stocks_refine_total.csv', encoding='latin1')
df4 = pd.read_csv('C:/example/test_sample.csv')
df = df.head(1000)

df2.drop(columns=["sector", "industry"], inplace=True)

df = df[['label_id', 'news_id', 'word', 'label']]
df['pk'] = 0
df['word2'] = 0
df['word2'] = df['word']

df2 = df2[['pk', 'symbol', 'name']]
df2['name_2'] = df2['name']
df['token'] = 0

# ORG단어와 종목명 소문자화
df['word'] = df['word'].str.lower()
df2['name_2'] = df2['name_2'].str.lower()
df['word2'] = df['word2'].str.lower()

# 정규식 전처리
df['word'] = df['word'].apply(lambda x: re.sub('[^a-zA-Z0-9]', ' ', str(x)).strip())
df2['name_2'] = df2['name_2'].apply(lambda x: re.sub('[^a-zA-Z0-9]', ' ', str(x)).strip())
df['word'] = df['word'].apply(lambda x: re.sub(r'\d+', ' ', str(x)).strip())
df2['name_2'] = df2['name_2'].apply(lambda x: re.sub(r'\d+', ' ', str(x)).strip())

# 토큰 단어 먼저 불용어한 후 토큰화하기 위해서 먼저 전처리
stop_words_token = {'bloomberg', 'inc',  'ltd', 'company','corporation','corp'}
# ORG 단어들을 토큰화
df['word'].replace(stop_words_token, '', regex=True, inplace=True)
df['token'] = df['word'].str.split(" ")

token_list = df['token']
# token 컬럼에 있는 띄어쓰기 지우기
token_list = token_list.apply(lambda x: [i for i in x if i != "" and i != " "])
token_list = token_list.apply(lambda x: x[:3])
df['token'] = token_list

# 불용어 리스트"
# ORG에 대한 불용어 리스트
stop_words_1 = {'bloomberg', 'inc',  'corp', 'ltd', 'company', 'corporation' ,'composite', 'financial','plc'}
# df2['name_2'](종목명)에 대한 불용어 리스트
stop_words_2 = {'inc',  'corp', 'ltd', 'company', 'corporation', 'fund','composite', 'financial', 'incorporated','index','plc'}

df['word'].replace(stop_words_1, '', regex=True, inplace=True)
df2['name_2'].replace(stop_words_2, '', regex=True, inplace=True)

df['word'] = df['word'].str.rstrip("co"" ")
df2['name_2'] = df2['name_2'].str.rstrip("co"" ")


df['word'] = df['word'].str.lstrip(" ")

l1 = df['word']
l2 = df2['name_2']

In [2]:
# ORG와 stocklist의 교집합이 존재시 맵핑
for p ,t in enumerate(token_list):
    for j ,k in enumerate(l2):
       k_list = k.split(" ")
       together = set(k_list)&set(t)
       if len(together)>=2:
            df['pk'][p] = df2['pk'][j]
            break
       elif len(together)>=3:
            df['pk'][p] = df2['pk'][j]
            break

C:\Users\q1035\anaconda3\envs\aip\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [3]:
# 유사도 계산
for i,j in enumerate(l1):
    for k,l in enumerate(l2):
        if fuzz.ratio(j, l)>80:
            df['pk'][i]=df2['pk'][k]
            break
        elif fuzz.ratio(j, l)>90:
            df['pk'][i]=df2['pk'][k]
            break
        elif j == l:
            df['pk'][i] = df2['pk'][k]
            break

C:\Users\q1035\anaconda3\envs\aip\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [4]:
# 대표단어 csv 파일 이용해서 맵핑
l3 = df['word2'].str.split(" ") # 전처리 안한 org단어
l4 = df3['name_a'].str.split(",") # 핵심단어로 정리된 stock_list
# 대표단어 csv파일에서 대표단어 수가 2개일 때 첫번째 단어가 있으면 넘어가서 두번째 자리에 다음 단어가 있는지 확인 후 맵핑
for i,j in enumerate(l3): # 전처리 안한 org단어
    for k,l in enumerate(l4): # 대표단어로 정리된 stock_list
        if len(l)==1:
            if l[0] in j:
                df['pk'][i]=df3['pk'][k]
                break
        elif len(l)==2: # 대표단어가 2개로 된 단어일 때
            if l[0] in j: # 대표단어의 첫번째 단어가 org단어에 있으면
                found = j.index(l[0]) # 대표단어의 첫번째 단어와 일치하는 org단어의 인덱스 위치
                try:
                    search = found+1 # stocklist의 첫번째 단어가 org에 포함됐을때 그 다음 단어
                    search_found = j[search] # org의 (+1을 한) 다음 단어에 해당
                    if l[1] == search_found:
                        df['pk'][i]=df3['pk'][k]
                        break
                except IndexError: continue

C:\Users\q1035\anaconda3\envs\aip\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\q1035\anaconda3\envs\aip\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [5]:
# 정말 stock이 아니라고 생각하는 단어가 포함되면 non처리
stop_word_3 = {'university', 'federal reserve','committee','national institute','department', 'bank of','united nation','commission','world bank'}
for i,j in enumerate(l1):
    for k,l in enumerate(stop_word_3):
        if l in j:
            df['pk'][i]=0
            break

C:\Users\q1035\anaconda3\envs\aip\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
# bank of america가 들어가면 맵핑
bank={'bank of america'}
for i,j in enumerate(l1):
    for k,l in enumerate(bank):
        if l in j:
            df['pk'][i]=860

C:\Users\q1035\anaconda3\envs\aip\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
df.loc[df['pk']==291,'pk']=0

In [8]:
df['word']=df4['word']

In [9]:
df.drop(columns=["word2", "token"], inplace=True)
df2.drop(columns=["name_2"], inplace=True)

In [10]:
print('전체개수 : ',len(df))
print('분류된개수 : ',len(df[df['pk']>0]))
print('분류된 비율 : ',len(df[df['pk']>0])/len(df))

전체개수 :  1000
분류된개수 :  249
분류된 비율 :  0.249


In [11]:
merge_left = pd.merge(df,df2, how='left', left_on='pk', right_on='pk')
merge_left

,label_id,news_id,word,label,pk,symbol,name
0,2516409,138767,Oaktree Capital Group LLC,ORG,5466,OAK^A,Oaktree Capital Group LLC 6
1,2516410,138767,DoubleLine Capital,ORG,0,NaN,NaN
2,2516413,138767,Federal Reserve,ORG,0,NaN,NaN
3,2516415,138767,ATMs,ORG,0,NaN,NaN
4,2516416,138767,Fed,ORG,0,NaN,NaN
...,...,...,...,...,...,...,...
995,4435835,282468,House of Commons,ORG,0,NaN,NaN
996,4435844,282468,cinemas,ORG,0,NaN,NaN
997,4435850,282468,"National Institute of Allergy, Infectious Dise...",ORG,0,NaN,NaN
998,4435851,282468,Morgan Stanley,ORG,4979,MS,Morgan Stanley


In [12]:
merge_left.to_csv("C:/example/finish21.csv")